In [1]:
from selenium import webdriver
from requests import session
from lxml.html import fromstring
from time import sleep
import csv,json,os,sys

Get Login to https://www.codewars.com/

In [2]:
# Read creds from config file
with open('config.json') as json_data_file:
    config = json.load(json_data_file)

email = config['email']
password = config['password']

Login Code

In [3]:
headers = {
    'authority': 'www.codewars.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://www.codewars.com',
    'referer': 'https://www.codewars.com/users/sign_in',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Microsoft Edge";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0',
}
sess = session()
sess.headers.update(headers)

In [4]:
# Get authenticity_token
login_page_resp = sess.get('https://www.codewars.com/users/sign_in')
login_page = fromstring(login_page_resp.text)
authenticity_token = login_page.xpath('//input[@name="authenticity_token"]/@value')[0]
authenticity_token

'75b90/lVDjhM0T8GEzF0gpE9BUolM59cjtjcyEP4gze4fsU5ZB7ZvR8pCV2OI9oV+IynEXuQn+gCCfGesAXp5w=='

In [5]:




data = {
  'utf8': '\u2713',
  'authenticity_token': str,
  'user[email]': str,
  'user[password]': str,
  'user[remember_me]': 'true'
}
data['authenticity_token'] = authenticity_token
data['user[email]'] = email
data['user[password]'] = password

login_resp = sess.post('https://www.codewars.com/users/sign_in', headers=headers, data=data)

login_resp

<Response [200]>

In [6]:
# write response html
with open('login.html', 'w', encoding='UTF-8') as f:
    f.write(login_resp.text)

In [7]:
login_resp.url

'https://www.codewars.com/dashboard'

Scrap Solutions

In [10]:
complete_sols_url = 'https://www.codewars.com/users/Pylogicx/completed_solutions'
complete_sols_resp = sess.get(complete_sols_url)
complete_sols_resp

<Response [200]>

In [12]:
soupTree = fromstring(complete_sols_resp.text)

In [18]:
codewar_domain = 'https://www.codewars.com'
kata_items_XPath = '//div/div[@class="list-item-solutions"]'
kata_items = soupTree.xpath(kata_items_XPath)
for item in kata_items:
    title_tag = item.find('./div[@class="item-title"]//a')
    title = title_tag.text
    kata_url = codewar_domain + title_tag.get('href')
    print(kata_url)

https://www.codewars.com/kata/58dced7b702b805b200000be
https://www.codewars.com/kata/59ca8246d751df55cc00014c
https://www.codewars.com/kata/58d248c7012397a81800005c
https://www.codewars.com/kata/54147087d5c2ebe4f1000805
https://www.codewars.com/kata/56f69d9f9400f508fb000ba7
https://www.codewars.com/kata/563e320cee5dddcf77000158
https://www.codewars.com/kata/55902c5eaa8069a5b4000083
https://www.codewars.com/kata/57f24e6a18e9fad8eb000296
https://www.codewars.com/kata/515e188a311df01cba000003
https://www.codewars.com/kata/57cfdf34902f6ba3d300001e
https://www.codewars.com/kata/55edaba99da3a9c84000003b
https://www.codewars.com/kata/57f6ad55cca6e045d2000627
https://www.codewars.com/kata/5b853229cfde412a470000d0
https://www.codewars.com/kata/58261acb22be6e2ed800003a
https://www.codewars.com/kata/5899642f6e1b25935d000161


In [22]:
# get kata description
kata_desc_resp = sess.get(kata_url)
# write resp to html
with open('kata_desc.html', 'w', encoding='UTF-8') as f:
    f.write(kata_desc_resp.text)

soupTree_desc = fromstring(kata_desc_resp.text)
# desc_XPath = '//h3[text()="Description:"]/following-sibling::div/p'
desc_XPath = '//div[@id="description"]/p'
desc = soupTree_desc.xpath(desc_XPath)
desc

[]

In [28]:
import requests

challenge = '5672a98bdbdd995fad00000f'
response = requests.get(f'https://www.codewars.com/api/v1/code-challenges/{challenge}')
json_chalange_resp = response.json()
description = json_chalange_resp['description']
description

'# Rock Paper Scissors\n\nLet\'s play! You have to return which player won! In case of a draw return `Draw!`.\n\n**Examples(Input1, Input2 --> Output):**\n\n```\n"scissors", "paper" --> "Player 1 won!"\n"scissors", "rock" --> "Player 2 won!"\n"paper", "paper" --> "Draw!"\n```\n\n![rockpaperscissors](http://i.imgur.com/aimOQVX.png)'